# Acerca del conjunto de datos

Cienmil imagenes generadas de forma aleatoria con entre 5 y 15 piezas.

Las imágenes fueron generadas usando 28 estilos de tableros de ajedrez y 32 estilos de piezas diferentes generando 896 combinaciones de estilo. 

Fueron generadas con este [generador de tableros digitales](https://github.com/koryakinp/chess-generator)

Las imagenes son originalmente de 400 x 400 pixeles. 

El conjunto de entrenamiento tiene 80000 imagenes y pruebas 20000 imagenes. 

La distribución de probabilidad es 

30% peón
20% alfil
20% Rey
20% Torre
10% Reina

El nombre del archivo fue etiquetado conforme a la notación Forsyth–Edwards Notation (FEN) que es una representación posicional de las piezas en el tablero:

![alt text](FEN_all_string.png)

![alt text](fen_one_rank.png)

# Librerías

In [11]:
from keras import backend as K
from keras import layers, models, optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.initializers import he_normal, lecun_normal
from tensorflow.keras.utils import plot_model
from math import ceil
from skimage import io, transform
from skimage.util.shape import view_as_blocks
from sklearn.model_selection import train_test_split, KFold
from tqdm import tqdm
import glob
import keras
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import re
import warnings
warnings.filterwarnings('ignore')


# Definición de variables para entrenamiento del modelo

In [12]:
BATCH_SIZE = 128 # Número de muestras procesadas en un lote durante el entrenamiento.
Epoch = 100 # Número de veces que el algoritmo trabajará a través de todo el conjunto de datos.
k_folds = 5 # Número de particiones para la validación cruzada.
PATIENCE = 5 # Número de épocas sin mejora después de las cuales el entrenamiento se detendrá.
SEED = 666 # Semilla para la generación de números aleatorios para reproducibilidad.
SQUARE_SIZE = 40 # Tamaño del lado del cuadrado, debe ser menor que 400/8=50.
test_size = 500 # Número de muestras en el conjunto de prueba.
train_size = 3000 # Número de muestras en el conjunto de entrenamiento.


# Inicialización de las semillas en numpy y tensorflow

In [13]:
random.seed(SEED)
from numpy.random import seed
seed(SEED)
import tensorflow
tensorflow.random.set_seed(SEED)

In [14]:
RUTA_DATOS = './data'
RUTA_IMAGENES_ENTRENAMIENTO = os.path.join(RUTA_DATOS, 'train')
RUTA_IMAGENES_PRUEBA = os.path.join(RUTA_DATOS, 'test')
def get_image_filenames(image_path, image_type):
    """
    Obtiene una lista de nombres de archivos de imágenes de un tipo específico en un directorio dado.

    Parámetros:
    - image_path (str): La ruta al directorio donde se encuentran las imágenes.
    - image_type (str): El tipo de archivo de las imágenes a buscar (ej. 'jpg', 'png').

    Retorna:
    - list: Una lista de rutas completas a los archivos que coinciden con el tipo de imagen especificado.
      Si el directorio no existe, devuelve None.

    Ejemplo:
    - get_image_filenames('/ruta/a/imagenes', 'jpg') -> Devuelve todas las rutas de archivos .jpg en el directorio especificado.
    """
    if(os.path.exists(image_path)):
        return glob.glob(os.path.join(image_path, '*.' + image_type))
    return None  # Explícitamente devuelve None si el directorio no existe.


In [15]:
datos = get_image_filenames(RUTA_IMAGENES_ENTRENAMIENTO, "jpeg")
test = get_image_filenames(RUTA_IMAGENES_PRUEBA, "jpeg")

random.shuffle(datos)
random.shuffle(test)

datos = datos[:train_size]
test = test[:test_size]

# Mostramos 10 como ejemplo

In [16]:
datos[0:10]

['./data\\train\\3Rk3-6p1-5p2-3pp1p1-1K6-8-R3b3-4B3.jpeg',
 './data\\train\\7N-5PK1-1n1b2b1-1p4k1-8-2n1p3-4N3-2b5.jpeg',
 './data\\train\\6r1-3P4-8-1K1pk2p-2r5-6r1-3p4-3q4.jpeg',
 './data\\train\\1b6-1K6-p7-8-Q3k2P-1R6-3B4-4nnq1.jpeg',
 './data\\train\\7Q-2RK1P2-1b6-5Pp1-2pr4-8-3k1RN1-bn2r3.jpeg',
 './data\\train\\1Kn4R-2p5-1N6-4pP2-1q2N3-8-5k2-5bN1.jpeg',
 './data\\train\\1K6-1R3rQ1-6k1-N7-1p4B1-r4p2-8-8.jpeg',
 './data\\train\\8-8-8-3K2kP-5bR1-8-1p1R1N2-1B6.jpeg',
 './data\\train\\6k1-8-5r2-b3p1B1-K7-4b3-7r-2Q2bBb.jpeg',
 './data\\train\\1b2b1kQ-8-8-1p2R3-2KR4-3bp3-2nN4-4r3.jpeg']

In [17]:
def fen_from_filename(filename):
    """
    Extrae el nombre base de un archivo sin su extensión desde una ruta de archivo completa.

    Parámetros:
    - filename (str): La ruta completa del archivo del cual se desea extraer el nombre base.

    Retorna:
    - str: El nombre del archivo sin la extensión.

    Ejemplo:
    - fen_from_filename('/ruta/al/archivo/ejemplo.txt') -> 'ejemplo'
    """
    base = os.path.basename(filename)  # Extrae el nombre del archivo con extensión desde la ruta completa.
    return os.path.splitext(base)[0]    # Elimina la extensión del nombre del archivo y devuelve el resultado.
print(fen_from_filename(datos[0]))


3Rk3-6p1-5p2-3pp1p1-1K6-8-R3b3-4B3


# Función para mostrar un rango de imágenes de un conjunto de datos

In [18]:
def muestra_rango_de_imagenes(datos, rango):
    """
    Muestra las primeras tres imágenes de una lista de rutas de archivos de imagen, 
    junto con sus nombres de archivo (sin extensiones) como títulos.

    Parámetros:
    - train (list): Lista de rutas completas a las imágenes a mostrar.
    - rango (range): ejemplo range(0, 3)

    Ejemplo:
    - mostrar_imagenes(['/ruta/a/imagen1.jpg', '/ruta/a/imagen2.jpg', '/ruta/a/imagen3.jpg'])
    """
    f, axarr = plt.subplots(1, 3, figsize=(120, 120))  # Configura un subplot con 3 ejes.

    for i in rango:
        base = os.path.basename(datos[i])             # Extrae el nombre del archivo con extensión.
        nombre_sin_extension = os.path.splitext(base)[0]  # Elimina la extensión del nombre del archivo.
        axarr[i].set_title(nombre_sin_extension, fontsize=70, pad=30)  # Establece el título del eje.
        axarr[i].imshow(mpimg.imread(datos[i]))       # Carga y muestra la imagen.
        axarr[i].axis('off')                          # Desactiva los ejes.

# De manera informativa, mostrar 3 imagenes

In [19]:
muestra_rango_de_imagenes(train, range(0,3))

NameError: name 'train' is not defined

# Funciones para tratar las cadenas FEN como one-hot y viceversa.

In [ ]:
import numpy as np
import re

def onehot_from_fen(fen):
    """
    Convierte una cadena FEN en una matriz one-hot que representa el estado del tablero de ajedrez.

    Parámetros:
    - fen (str): Cadena FEN que describe la disposición de las piezas en un tablero de ajedrez.

    Retorna:
    - np.ndarray: Una matriz one-hot donde cada fila representa una pieza en el tablero y cada columna
                  representa un tipo de pieza o un espacio vacío.

    Detalles:
    - '12345678' se utilizan para representar el número de espacios vacíos consecutivos.
    - 'piece_symbols' debe ser una lista previamente definida con los símbolos de las piezas correspondientes.
    """
    piece_symbols = 'prbnkqPRBNKQ'
    eye = np.eye(13)  # Crea una matriz identidad para representar las piezas y espacios vacíos.
    output = np.empty((0, 13))  # Inicializa la matriz de salida.
    fen = re.sub('[-]', '', fen)  # Elimina guiones, que no son necesarios en la representación.

    for char in fen:
        if char in '12345678':
            output = np.append(output, np.tile(eye[12], (int(char), 1)), axis=0)  # Espacios vacíos.
        else:
            idx = piece_symbols.index(char)  # Encuentra el índice de la pieza en la lista.
            output = np.append(output, eye[idx].reshape((1, 13)), axis=0)  # Añade la pieza a la matriz.

    return output

def fen_from_onehot(one_hot):
    """
    Convierte una matriz one-hot en una cadena FEN que representa el estado del tablero de ajedrez.

    Parámetros:
    - one_hot (np.ndarray): Matriz one-hot donde cada fila representa una pieza en el tablero y cada columna
                            representa un tipo de pieza o un espacio vacío.

    Retorna:
    - str: Una cadena FEN que describe la disposición de las piezas en un tablero de ajedrez.

    Detalles:
    - 'piece_symbols' debe ser una lista previamente definida con los símbolos de las piezas correspondientes.
    """
    piece_symbols = 'prbnkqPRBNKQ'
    output = ''
    for j in range(8):
        for i in range(8):
            if one_hot[j][i] == 12:
                output += ' '  # Representa un espacio vacío.
            else:
                output += piece_symbols[one_hot[j][i]]  # Añade el símbolo de la pieza.
        if j != 7:
            output += '-'  # Añade un guión entre las filas.

    for i in range(8, 0, -1):
        output = output.replace(' ' * i, str(i))  # Compacta espacios vacíos en números.

    return output


In [ ]:
# Ejemplo de onehot from fen limitado a 10 filas
onehot_from_fen('1r6-8-8-5Q2-1K4r1-6P1-Pb6-3k4')[0:8]
# Recordar que el orden es: prbnkqPRBNKQ
# La primera fila dice 1 vacia, luego rook, luego 6 vacias. 
# Esto está representado en one-hot de la siguiente manera
# Primera fila, la ultima columna representa casilla vacía y está en 1
# La fila 2, la columna 2, es la rook negra y está en 1
# Luego 6 filas que también representan casilla vacía

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

# Función para procesar imágenes de tableros de ajedrez, que regresa los 64 cuadros

In [ ]:
from skimage import io, transform
from skimage.util import view_as_blocks

def process_image(img):
    """
    Procesa una imagen reduciéndola a un tamaño especificado y dividiéndola en cuadrados más pequeños.

    Parámetros:
    - img (str): Ruta al archivo de imagen que se va a procesar.

    Retorna:
    - np.ndarray: Un arreglo numpy que contiene 64 tiles (cuadrados) de la imagen, cada uno con
                  dimensiones (SQUARE_SIZE, SQUARE_SIZE, 3), donde 3 representa los canales de color RGB.

    Detalles:
    - 'SQUARE_SIZE' debe ser definida externamente y representa el tamaño de cada cuadrado en el que se divide la imagen.
    - La imagen es redimensionada a un tamaño de (SQUARE_SIZE*8, SQUARE_SIZE*8) antes de ser dividida.
    - Esta función es útil para preparar imágenes para su análisis o procesamiento en tareas que requieren
      datos en forma de cuadrícula o matriz, como el análisis de tableros de juegos.
    """
    new_image_size = SQUARE_SIZE * 8  # Calcula el nuevo tamaño deseado de la imagen.
    square_size = SQUARE_SIZE  # Tamaño de cada cuadrado en el que se dividirá la imagen.
    img_read = io.imread(img)  # Lee la imagen desde la ruta especificada.
    # Redimensiona la imagen al tamaño deseado.
    img_read = transform.resize(img_read, (new_image_size, new_image_size), mode='constant')
    # Divide la imagen en bloques más pequeños de tamaño (square_size, square_size, 3).
    tiles = view_as_blocks(img_read, block_shape=(square_size, square_size, 3))
    # Elimina una dimensión redundante que podría haber sido creada durante el proceso de división.
    tiles = tiles.squeeze(axis=2)
    # Reorganiza las baldosas para que estén en una sola dimensión con las dimensiones requeridas por tile.
    return tiles.reshape(64, square_size, square_size, 3)


# Función train_gen
Esta función es un generador que produce lotes de datos de entrenamiento a partir de un conjunto de características, utilizando una codificación one-hot y procesamiento de imágenes.

In [ ]:
def train_gen(features_paths, batch_size):
    """
    Genera lotes de datos de entrenamiento a partir de un conjunto de rutas de imágenes.
    Etiquetandolas con la codificación one-hot

    Parámetros:
    - features_paths (list): Lista de rutas de imágenes.
    - batch_size (int): Tamaño del lote de imágenes a procesar.

    Yields:
    - tuple: Tupla que contiene dos arrays numpy, uno para las imágenes procesadas (X) y otro para las etiquetas (Y),
             cada uno codificado como one-hot.
    """
    i = 0
    while True:
        batch_x, batch_y = [], []
        for _ in range(batch_size):
            if i == len(features_paths):
                i = 0
                random.shuffle(features_paths)
            img = str(features_paths[i])
            y = onehot_from_fen(fen_from_filename(img))
            x = process_image(img)
            batch_x.extend(x)
            batch_y.extend(y)
            i += 1
        yield (np.array(batch_x), np.array(batch_y))

# Función pred gen
Esta función es un generador que produce datos procesados de imágenes para predicciones.

In [ ]:
def pred_gen(features_paths, batch_size):
    """
    Genera lotes de imágenes procesadas para predicción.

    Parámetros:
    - features_paths (list): Lista de rutas de imágenes.
    - batch_size (int): Tamaño del lote de imágenes a procesar.

    Yields:
    - np.ndarray: Array de imágenes procesadas.
    """
    i = 0
    while i < len(features_paths):
        batch_images = [process_image(features_paths[j]) for j in range(i, min(i + batch_size, len(features_paths)))]
        i += batch_size
        yield np.array(batch_images)

# Función de callbacks documentada

In [ ]:
def get_callbacks(model_name, patient):
    """
    Configura y retorna una lista de callbacks útiles para el entrenamiento de modelos en Keras.

    Parámetros:
    - model_name (str): Ruta y nombre del archivo donde se guardará el modelo con mejor rendimiento.
    - patient (int): Número de épocas sin mejora en la pérdida de validación después de las cuales se tomarán medidas.

    Retorna:
    - list: Lista de objetos callback configurados para ser usados durante el entrenamiento.

    Callbacks configurados:
    - EarlyStopping: Detiene el entrenamiento cuando una métrica monitoreada ha dejado de mejorar.
      * monitor='val_loss': La métrica a monitorear, en este caso, la pérdida de validación.
      * patience=patient: El número de épocas para esperar después de una mejora antes de detener el entrenamiento.
      * mode='min': Establece que la métrica monitoreada debe minimizarse (pérdida de validación menor es mejor).
      * verbose=1: Habilita la salida detallada de mensajes (1) para visualizar el progreso en la consola.

    - ReduceLROnPlateau: Reduce la tasa de aprendizaje cuando una métrica de rendimiento se ha estancado.
      * monitor='val_loss': La métrica a monitorear.
      * factor=0.5: Factor por el cual se reducirá la tasa de aprendizaje.
      * patience=patient / 2: Número de épocas para esperar antes de reducir la tasa de aprendizaje.
      * min_lr=0.000001: El límite inferior de la tasa de aprendizaje.
      * verbose=1: Habilita mensajes detallados.
      * mode='min': Busca minimizar la métrica monitoreada.

    - ModelCheckpoint: Guarda el modelo después de cada época solo si es el mejor encontrado hasta el momento en términos de pérdida de validación.
      * filepath=model_name: Ruta donde se guardará el modelo.
      * monitor='val_loss': Métrica para determinar el mejor modelo.
      * verbose=1: Habilita mensajes detallados.
      * save_best_only=True: Guarda solo el modelo que tiene la mejor métrica de 'val_loss'.
      * mode='min': Busca minimizar la métrica monitoreada.

    Ejemplo de uso:
    callbacks = get_callbacks('modelo_mejor.h5', 10)
    model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=callbacks)
    """
    # Configuración de EarlyStopping
    ES = EarlyStopping(monitor='val_loss', patience=patient, mode='min', verbose=1)
    # Configuración de ReduceLROnPlateau
    RR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patient / 2), min_lr=0.000001, verbose=1, mode='min')
    # Configuración de ModelCheckpoint
    MC = ModelCheckpoint(filepath=model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    return [ES, RR, MC]

# Funcionamiento de la Pérdida por Entropía Cruzada Categórica Ponderada
La entropía cruzada categórica es una medida comúnmente utilizada para evaluar el error entre las probabilidades predichas y las etiquetas verdaderas en problemas de clasificación. La versión ponderada modifica esta medida para dar más o menos importancia a ciertas clases durante el entrenamiento del modelo:

- Normalización y Escalado: Las predicciones (y_pred) se normalizan para asegurar que las probabilidades de cada muestra sumen 1, ajustándolas según los pesos de cada clase.
- Estabilidad Numérica: Se utiliza K.clip para evitar valores de logaritmo de cero, que resultarían en NaN o Inf, lo cual podría desestabilizar el entrenamiento del modelo.
- Aplicación de Pesos: Los pesos se aplican directamente en el cálculo del logaritmo de las predicciones, lo cual ajusta el impacto de las predicciones incorrectas según la importancia de cada clase. Así, los errores en clases con mayor peso tienen un impacto proporcionalmente mayor en la función de pérdida.

Esta funcionalidad es especialmente útil en datasets desbalanceados, donde algunas clases son menos frecuentes pero más críticas. Al ajustar los pesos, se puede guiar al modelo para que preste más atención a estas clases menos representadas pero importantes.

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    Devuelve una función de pérdida de entropía cruzada categórica ponderada personalizada para usar en entrenamientos de modelos de Keras.

    Parámetros:
    - weights (np.array): Un arreglo de numpy con forma (C,) donde C es el número de clases. 
                          Cada elemento del arreglo asigna un peso a la correspondiente clase para calcular la pérdida.

    Uso:
    - weights = np.array([0.5, 2, 10]) # Clase 1 con peso 0.5, clase 2 con el doble del peso normal, clase 3 con 10 veces el peso.
    - loss = weighted_categorical_crossentropy(weights)
    - model.compile(loss=loss, optimizer='adam')

    Retorna:
    - loss (function): Una función que calcula la entropía cruzada categórica ponderada entre `y_true` y `y_pred`.

    Ejemplo de uso:
    model.compile(loss=weighted_categorical_crossentropy(np.array([1, 2, 0.5])), optimizer='adam')
    """
    
    weights = K.variable(weights)  # Convierte los pesos a una variable de Keras para su uso en cálculos de tensor.

    def loss(y_true, y_pred):
        # Escala las predicciones para que la suma de probabilidades de cada muestra sea 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # Limita los valores predichos para evitar NaN's e Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # Calcula la pérdida usando los pesos asignados a cada clase
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss


# Definición del modelo

## Descripción del Modelo de Red Neuronal Convolucional

### Arquitectura del Modelo

La arquitectura del modelo está diseñada para procesar imágenes mediante una serie de capas que extraen características y aplican regularización para mejorar la generalización:

1. **Capas Conv2D**: 
   - Tres capas convolucionales con 32 filtros de tamaño 3x3 y activación ReLU. Estas capas extraen características visuales básicas como bordes y texturas.

2. **Capas de Dropout**: 
   - Capas que aplican un dropout del 20% después de cada capa convolucional para reducir el riesgo de sobreajuste, ayudando al modelo a generalizar mejor a nuevos datos.

3. **Capa MaxPooling2D**: 
   - Una capa de agrupación máxima con un tamaño de 2x2 que reduce la dimensionalidad de las características, conservando las características más prominentes.

4. **Capa Flatten**: 
   - Esta capa aplana las características multidimensionales en un vector unidimensional, preparándolas para el procesamiento en capas densas.

5. **Capas Dense**: 
   - Una capa densa de 128 neuronas con activación ReLU seguida de una capa de salida con 13 neuronas y activación softmax, que clasifica las imágenes en una de las 13 categorías basadas en las características extraídas.

### Compilación del Modelo

- El modelo utiliza la entropía cruzada categórica ponderada como función de pérdida, con un ajuste dinámico de la tasa de aprendizaje y la métrica de precisión para evaluar el rendimiento.


In [ ]:
import numpy as np
from keras import models, layers
from keras.optimizers import Nadam

class ChessFENClassifier:
    """
    Clasificador de imágenes de ajedrez para convertir imágenes cuadradas en cadenas FEN utilizando una red neuronal convolucional.

    Atributos:
    - image_size (int): Tamaño de las imágenes (ancho y alto) que acepta el modelo.

    Métodos:
    - __init__(self, image_size): Constructor de la clase que inicializa el tamaño de la imagen.
    - build_model(self): Construye y compila el modelo de red neuronal.
    """

    def __init__(self, image_size):
        """
        Inicializa el clasificador con el tamaño especificado de las imágenes.

        Parámetros:
        - image_size (int): Tamaño de las imágenes cuadradas.
        """
        self.image_size = image_size
        self.model = self.build_model()

    def build_model(self):
        """
        Construye y compila el modelo de red neuronal utilizando la arquitectura CNN con Keras.

        Retorna:
        - model (keras.Model): Modelo de red neuronal convolucional compilado.
        """
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(self.image_size, self.image_size, 3)))
        model.add(layers.Dropout(0.2))
        model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(layers.Dropout(0.2))
        model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(layers.Dropout(0.2))
        model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal'))
        model.add(layers.Dropout(0.2))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
        model.add(layers.Dropout(0.2))
        model.add(layers.Dense(13, activation='softmax', kernel_initializer='lecun_normal'))

        # Cálculo de los pesos para la pérdida por entropía cruzada categórica ponderada
        # 30% peón 20% alfil 20% Rey 20% Torre 10% Reina
        weights = np.array([1/(0.30*4), 1/(0.20*4), 1/(0.20*4), 1/(0.20*4), 1/1, 1/(0.10*4),
                            1/(0.30*4), 1/(0.20*4), 1/(0.20*4), 1/(0.20*4), 1/1, 1/(0.10*4), 1/(64-10)])
        model.compile(loss=weighted_categorical_crossentropy(weights), optimizer=Nadam(), metrics=['accuracy'])

        return model


# Entrenamiento del modelo

In [ ]:
classifier = ChessFENClassifier(SQUARE_SIZE)
model = classifier.model
model

NameError: name 'ChessFENClassifier' is not defined